In [1]:
%matplotlib inline
import os
os.chdir('U:\\GitHubClones\\SeniorDesign')
os.getcwd()
import Excel
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import OrderedDict
import re
from datetime import datetime

In [2]:
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 1000)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
path = Excel.directory + "\\" + Excel.shelf + "\\" + Excel.dealName + "\\" + Excel.reportList[8]
print(path)

U:\AutoLoanTapes\Santander Drive Auto Receivables Trust\SDART 2017-1\2017\09 September\2017.10.16_Santander Drive Auto Receivables Trust 2017-1 Data Tape.csv


In [4]:
df = pd.read_csv(path, memory_map=True, low_memory=False)
len(df.index)


60291

In [8]:
states = list(df['obligorGeographicLocation'])
states
df['states'] = states
df.head()

,assetTypeNumber,assetNumber,reportingPeriodBeginningDate,reportingPeriodEndingDate,originatorName,originationDate,originalLoanAmount,originalLoanTerm,loanMaturityDate,originalInterestRatePercentage,interestCalculationTypeCode,originalInterestRateTypeCode,originalInterestOnlyTermNumber,originalFirstPaymentDate,underwritingIndicator,gracePeriodNumber,paymentTypeCode,subvented,vehicleManufacturerName,vehicleModelName,vehicleNewUsedCode,vehicleModelYear,vehicleTypeCode,vehicleValueAmount,vehicleValueSourceCode,obligorCreditScoreType,obligorCreditScore,obligorIncomeVerificationLevelCode,obligorEmploymentVerificationCode,coObligorIndicator,paymentToIncomePercentage,obligorGeographicLocation,assetAddedIndicator,remainingTermToMaturityNumber,reportingPeriodModificationIndicator,servicingAdvanceMethodCode,reportingPeriodBeginningLoanBalanceAmount,nextReportingPeriodPaymentAmountDue,reportingPeriodInterestRatePercentage,nextInterestRatePercentage,servicingFeePercentage,servicingFlatFeeAmount,otherServicerFeeRetainedByServicer,otherAssessedUncollectedServicerFeeAmount,scheduledInterestAmount,scheduledPrincipalAmount,otherPrincipalAdjustmentAmount,reportingPeriodActualEndBalanceAmount,reportingPeriodScheduledPaymentAmount,totalActualAmountPaid,actualInterestCollectedAmount,actualPrincipalCollectedAmount,actualOtherCollectedAmount,servicerAdvancedAmount,interestPaidThroughDate,zeroBalanceEffectiveDate,zeroBalanceCode,currentDelinquencyStatus,primaryLoanServicerName,mostRecentServicingTransferReceivedDate,assetSubjectDemandIndicator,assetSubjectDemandStatusCode,repurchaseAmount,DemandResolutionDate,repurchaserName,repurchaseReplacementReasonCode,chargedoffPrincipalAmount,recoveredAmount,modificationTypeCode,paymentExtendedNumber,repossessedIndicator,repossessedProceedsAmount,states
0,SDART0201700001,"=""6345119""",09-01-2017,09-30-2017,SC,10/2013,23260.03,60,01/2019,0.1194,1,1,-,12/2013,True,2,2,0,Chevrolet,Traverse,2,2010,3,17589.00,98,Bureau,671,3,2,False,0.187788,AZ,-,17,False,1,8661.89,578.85,0.1194,0.1194,0.03,-,-,0.00,69.0948,447.6052,0.0,8213.43,516.70,519.30,70.84,448.46,0.0,-,09-08-2017,-,-,0,SC,-,False,-,-,-,-,-,-,-,-,0,False,-,AZ
1,SDART0201700001,"=""634535""",09-01-2017,09-30-2017,SC,03/2012,12031.00,72,03/2018,0.1800,1,1,-,04/2012,True,1,2,0,MAZDA,Mazda3-4 Cyl.,2,2010,1,13575.00,98,Bureau,-,2,2,False,0.069108,TX,-,7,False,1,1793.05,279.10,0.1800,0.1800,0.03,-,-,0.00,25.3291,249.0709,0.0,1541.35,274.40,280.00,28.30,251.70,0.0,-,09-22-2017,-,-,0,SC,-,False,-,-,-,-,-,-,-,-,0,False,-,TX
2,SDART0201700001,"=""6345360""",09-01-2017,09-30-2017,SC,11/2013,21332.31,72,11/2019,0.1810,1,1,-,12/2013,True,1,2,0,Dodge,Grand Caravan Passenger,2,2010,1,15209.00,3,Bureau,560,2,2,False,0.109852,WA,-,27,False,1,12159.23,1738.74,0.1810,0.1810,0.03,-,-,122.80,159.7583,327.9917,0.0,12011.55,487.75,491.37,343.69,147.68,0.0,-,09-14-2017,-,-,30,SC,-,False,-,-,-,-,-,-,-,-,0,False,-,WA
3,SDART0201700001,"=""6345391""",09-01-2017,09-30-2017,SC,11/2013,24544.23,72,11/2019,0.0855,1,1,-,12/2013,True,1,2,0,TOYOTA,Sienna-V6,2,2006,1,16550.00,98,Bureau,712,2,2,False,0.007974,FL,-,27,False,1,10524.88,303.63,0.0855,0.0855,0.03,-,-,0.00,76.2232,360.7368,0.0,10101.30,436.96,500.00,76.42,423.58,0.0,-,09-01-2017,-,-,0,SC,-,False,-,-,-,-,-,-,-,-,0,False,-,FL
4,SDART0201700001,"=""634551""",09-01-2017,09-30-2017,SC,03/2012,23589.10,72,09/2018,0.1710,1,1,-,04/2012,True,1,2,0,CHEVROLET,Sonic-4 Cyl.,1,2012,1,16607.31,98,Bureau,631,3,3,False,0.074936,IL,-,13,False,1,0.00,5190.31,0.0000,0.0000,0.03,-,-,1038.73,46.4841,479.5959,0.0,0.00,526.08,0.00,0.00,0.00,0.0,-,10-07-2016,05/2017,4,243,SC,-,False,-,-,-,-,-,0.00,0.00,-,0,True,4612.00,IL


In [100]:
WA = lambda x: np.average(x, axis=0, weights = df.loc[x.index, 'reportingPeriodActualEndBalanceAmount']) #have to calculate weighted average values before performing loan Cut

In [19]:
df['LTV'] = (df['originalLoanAmount']/df['vehicleValueAmount'])*100 #Loan to value ratio
df['pctDeal'] = (df['reportingPeriodActualEndBalanceAmount']/(df['reportingPeriodActualEndBalanceAmount'].sum()))*100 #Individual Loan as a Percent of Current month Balance
df['totalCollected'] = df['actualInterestCollectedAmount'] + df['actualPrincipalCollectedAmount'] + df['actualOtherCollectedAmount']
df['principalPaid'] = df['originalLoanAmount'] - df['reportingPeriodActualEndBalanceAmount']

print(df['pctDeal'].head())
df['LTV'].head()

0    0.000844
1    0.000158
2    0.001234
3    0.001038
4    0.000000
Name: pctDeal, dtype: float64


0    132.241913
1     88.626151
2    140.261095
3    148.303505
4    142.040463
Name: LTV, dtype: float64

In [21]:
preDefault = df[['assetNumber', 'zeroBalanceCode', 'originalLoanAmount', 'reportingPeriodActualEndBalanceAmount', 'principalPaid','totalActualAmountPaid','totalCollected']]
preDefault.head(10)

,assetNumber,zeroBalanceCode,originalLoanAmount,reportingPeriodActualEndBalanceAmount,principalPaid,totalActualAmountPaid,totalCollected
0,"=""6345119""",-,23260.03,8213.43,15046.60,519.30,519.30
1,"=""634535""",-,12031.00,1541.35,10489.65,280.00,280.00
2,"=""6345360""",-,21332.31,12011.55,9320.76,491.37,491.37
3,"=""6345391""",-,24544.23,10101.30,14442.93,500.00,500.00
4,"=""634551""",4,23589.10,0.00,23589.10,0.00,0.00
5,"=""6345523""",-,31254.56,23673.97,7580.59,0.00,0.00
6,"=""6345572""",-,23467.13,17214.66,6252.47,580.00,580.00
7,"=""6345618""",-,26589.78,15256.10,11333.68,0.00,0.00
8,"=""6345794""",-,28291.04,22188.77,6102.27,1233.52,1233.52
9,"=""6346049""",-,24623.30,17137.27,7486.03,570.00,570.00


In [17]:
for i in range(0,10):
    path = Excel.directory + "\\" + Excel.shelf + "\\" + Excel.dealName + "\\" + Excel.reportList[i]
    df = pd.read_csv(path, memory_map=True, low_memory=False)
    
    df['LTV'] = (df['originalLoanAmount']/df['vehicleValueAmount'])*100 #Loan to value ratio
    df['pctDeal'] = (df['reportingPeriodActualEndBalanceAmount']/(df['reportingPeriodActualEndBalanceAmount'].sum()))*100
    totalBalAmount = df['reportingPeriodActualEndBalanceAmount'].sum()
    
    
    df = df[~df.zeroBalanceCode.isin(['-'])] #taking out all of the performing loans so far
    df['zeroBalanceCode'] = pd.to_numeric(df['zeroBalanceCode'], errors='coerce')
    df.loc[df['zeroBalanceCode'] > 1] = 0
    

'\nfor i in range(0,10):\n    path = Excel.directory + "\\" + Excel.shelf + "\\" + Excel.dealName + "\\" + Excel.reportList[i]\n    df = pd.read_csv(path, memory_map=True, low_memory=False)\n\n'

### Converting Zero Balance Codes

In [22]:
df = df[~df.zeroBalanceCode.isin(['-'])]
print(len(df.index))
print(df['zeroBalanceCode'].unique())
df['zeroBalanceCode'] = pd.to_numeric(df['zeroBalanceCode'], errors='coerce')
print(df['zeroBalanceCode'].dtype)
df['zeroBalanceCode'].head()

2862
['4' '1' '3']
int64


4      4
43     1
73     4
95     4
136    4
Name: zeroBalanceCode, dtype: int64

In [23]:
df.loc[df['zeroBalanceCode'] > 1] = 0 # 1 means the loan has paid down, 0 means the loan has defaulted in some way
df['zeroBalanceCode'].unique()
df['zeroBalanceCode'].head()

4      0
43     1
73     0
95     0
136    0
Name: zeroBalanceCode, dtype: int64

#### Changing Values to numeric

In [24]:
df['obligorCreditScore'] = pd.to_numeric(df['obligorCreditScore'], errors='coerce')
df['subvented'] = pd.to_numeric(df['subvented'], errors='coerce')

### Bins For Cutting Data

In [25]:
FicoBins = [-1, 440, 500, 560, 620, 680, 740, 800, 1000]
APRBins = [0, .02 ,.04, .06, .08, .1, .12, .14, .16, 1000]
LTVBins = [-1, .7, .8, .9, 1, 1.1, 1.2, 1.3, 1.4, 1000]
PTIBins = [-1, .02, .04, .06, .08, .1, .12, .14, .16, 1000]
ZeroBalanceCodeBins = [-1, 0, 1]
SubventedCodeBins = [-1, 0, 1, 2, 3]
IncomeVerificationCodeBins = [0, 1, 2, 3, 4, 5]

In [26]:
FicoGroup = ['[-1, 440)', '[440, 500)', '[500, 560)', '[560, 620)', '[620, 680)', '[680, 740)', '[740, 800)', '[800, 1000)']
APRGroup = ['[0.0, 0.02)', '[0.02, 0.04)', '[0.04, 0.06)', '[0.06, 0.08)', '[0.08, 0.1)', '[0.1, 0.12)', '[0.12, 0.14)', '[0.14, 0.16)', '[0.16, 1000.0)']
PTIGroup = ['[-1.0, 0.02)', '[0.02, 0.04)', '[0.04, 0.06)', '[0.06, 0.08)', '[0.08, 0.1)', '[0.1, 0.12)', '[0.12, 0.14)', '[0.14, 0.16)', '[0.16, 1000.0)']
LTVGroup = ['[-1.0, 0.7)', '[0.7, 0.8)', '[0.8, 0.9)', '[0.9, 1.0)', '[1.0, 1.1)', '[1.1, 1.2)', '[1.2, 1.3)', '[1.3, 1.4)', '[1.4, 1000.0)']
ZeroBalanceCodeGroup = ['Default', 'Paid Down']
SubventedCodeGroup = ['None', 'Rate', 'Cash', 'Other']
IncomeVerificationCodeGroup = ['None', 'Stated', 'Stated & Verified', 'Extra Verfied', '2 Years Verified']

### Cutting the Data

In [36]:
df['Fico Group'] = pd.cut(df['obligorCreditScore'], FicoBins, right=True, labels=FicoGroup)
df['APR Group'] = pd.cut(df['originalInterestRatePercentage'], APRBins, right=True, labels=APRGroup)
df['PTI Group'] = pd.cut(df['paymentToIncomePercentage'], PTIBins, right=True, labels=PTIGroup)
df['Zero Balance Code Group'] = pd.cut(df['zeroBalanceCode'], ZeroBalanceCodeBins, right=True, labels=ZeroBalanceCodeGroup)
df['Subvented Code Group'] = pd.cut(df['subvented'], SubventedCodeBins, right=True, labels=SubventedCodeGroup)
df['Income Verification Code Group'] = pd.cut(df['obligorIncomeVerificationLevelCode'], IncomeVerificationCodeBins, right=True, labels=IncomeVerificationCodeGroup)


#### Adding No Score Category

In [37]:
df['Fico Group'] = df['Fico Group'].cat.add_categories('No Score').fillna('No Score')

In [54]:
a = OrderedDict([
    ('assetNumber', ['count']),
    ('originalInterestRatePercentage', ['mean']),
    ('chargedoffPrincipalAmount', ['mean']),
    ('totalActualAmountPaid', ['mean'])
])

In [41]:
dfComp = df[['originalInterestRatePercentage', 'Zero Balance Code Group', 'Fico Group']]
dfComp
df['Zero Balance Code Group'].value_counts()

Default      2083
Paid Down     779
Name: Zero Balance Code Group, dtype: int64

In [42]:
dfComp.groupby('Zero Balance Code Group').count()

,originalInterestRatePercentage,Fico Group
Zero Balance Code Group,,
Default,2083,2083
Paid Down,779,779


In [43]:
df[['assetNumber']].groupby([df['Fico Group'], df['Zero Balance Code Group']]).count()

assetNumber
Fico Group  Zero Balance Code Group             
[-1, 440)   Default                       2083.0
            Paid Down                        NaN
[440, 500)  Default                          NaN
            Paid Down                       21.0
[500, 560)  Default                          NaN
            Paid Down                      104.0
[560, 620)  Default                          NaN
            Paid Down                      279.0
[620, 680)  Default                          NaN
            Paid Down                      197.0
[680, 740)  Default                          NaN
            Paid Down                       46.0
[740, 800)  Default                          NaN
            Paid Down                        8.0
[800, 1000) Default                          NaN
            Paid Down                       10.0
No Score    Default                          NaN
            Paid Down                      114.0

In [57]:
df[['assetNumber', 'originalInterestRatePercentage']].groupby([ df['Fico Group'], df['Zero Balance Code Group']]).agg(a)

KeyError: 'chargedoffPrincipalAmount'